In [2]:
!pip install face_recognition

from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive/', force_remount=True)

import os
os.chdir(os.path.join(os.getcwd(), 'drive', 'MyDrive', 'Colab Notebooks', 'DeepFake Detector'))
print(f'current cwd = {os.getcwd()}')

Mounted at /content/drive/
current cwd = /content/drive/MyDrive/Colab Notebooks/DeepFake Detector


In [3]:
import os

from tqdm import tqdm
from PIL import Image
from heapq import nsmallest
from math import ceil
from random import randint, seed
from time import time

In [4]:
from pipeline import *

seed(0)

real_video_dir_path = os.path.join('.', 'Celeb-DF-v2', 'Celeb-real')
fake_video_dir_path = os.path.join('.', 'Celeb-DF-v2', 'Celeb-synthesis')

video_dir_path = real_video_dir_path
video_listdir = os.listdir(video_dir_path)

In [5]:
for video_name in tqdm(video_listdir):
    start_time = time()
    video = os.path.join(video_dir_path, video_name)

    frames = get_every_frame(video, interval=1)
    face_coordinates = get_face_locations(frames)
    centroid = get_centroid(face_coordinates)
    dist_from_centroid = get_distance_from_centroid(centroid,
                                                    face_coordinates)

    movement_thershold = nsmallest(4, set(filter(bool, dist_from_centroid)))[-1]

    stable_faces = get_stable_faces(movement_thershold,
                                dist_from_centroid, 
                                face_coordinates, 
                                frames, 
                                zoomed=False)

    number_of_frames_w_faces = len(stable_faces)
    sections = 6
    differencing_interval = ceil(number_of_frames_w_faces/sections)

    differenced_face = difference(stable_faces, interval=differencing_interval)
    Image.fromarray(differenced_face, mode='RGB').save(
        os.path.join(f'{video_dir_path}-diff', f'{video_name}.png'),'PNG')

    average_face = average(stable_faces)
    Image.fromarray(average_face, mode='RGB').save(
        os.path.join(f'{video_dir_path}-avg', f'{video_name}.png'),'PNG')

    random_face = stable_faces[randint(0, number_of_frames_w_faces-1)]
    Image.fromarray(random_face, mode='RGB').save(
        os.path.join(f'{video_dir_path}-rnd', f'{video_name}.png'),'PNG')

    # print(f'\n{video_name} pre-processed in {time()-start_time:.2f}s')

  1%|          | 6/590 [11:12<18:48:29, 115.94s/it]

KeyboardInterrupt: ignored